## Handling asynchronous operations

In [1]:
import nest_asyncio
nest_asyncio.apply()

## Set up Qdrant vector database

In [2]:
import qdrant_client
collection_name = "chat_with_docs"
client = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)

## Read the documents

In [3]:
from llama_index.core import SimpleDirectoryReader

input_dir = "./docs"

loader = SimpleDirectoryReader(
    input_dir=input_dir,
    required_exts=[".pdf"],
    recursive=True
)

docs = loader.load_data()

## A function to index data

In [4]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, ServiceContext, StorageContext

def create_index(documents):

    vector_store = QdrantVectorStore(client=client,
                                     collection_name=collection_name)
    
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    index = VectorStoreIndex.from_documents(documents,
                                            storage_context=storage_context)
    
    return index

## Load the embedding model and index data

In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5",
                                   trust_remote_code=True)

Settings.embed_model = embed_model

index = create_index(docs)

/Users/manideepbangaru/Documents/Learnings/RAGs_deepdive/ragEnv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load the LLM

In [6]:
from llama_index.llms.openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [12]:
llm = OpenAI(
    api_key=openai_api_key,
    model = "gpt-4o-mini",
    request_timeout=120
)

Settings.llm = llm

## Define the prompt template

In [13]:
from llama_index.core import PromptTemplate

template = """Context information is below:
              ---------------------
              {context_str}
              ---------------------
              Given the context information above I want you to think
              step by step to answer the query in a crisp manner,
              incase you don't know the answer say 'I don't know!'
            
              Query: {query_str}
        
              Answer:"""

qa_prompt_tmpl = PromptTemplate(template)

## Re-ranking

In [14]:
from llama_index.core.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-2-v2", 
    top_n=3
)

## Query the document

In [15]:
query_engine = index.as_query_engine(similarity_top_k=10,
                                     node_postprocessors=[rerank])

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

In [16]:
response = query_engine.query("What exactly is DSPy?")

## Display the output

In [20]:
from IPython.display import display, Markdown
display(Markdown(str(response)))

DSPy is a programming framework designed to streamline the process of interacting with language models (LMs) by using natural language signatures. These signatures serve as typed declarations that specify the input and output requirements for text transformations, allowing users to define what a function should do without detailing how to prompt the LM. DSPy compiles these signatures into adaptive prompts or fine-tunes, enhancing the efficiency and effectiveness of NLP tasks. It includes various modules that implement sophisticated prompting techniques, enabling the creation of self-improving multi-stage NLP systems with smaller and more efficient LMs. Overall, DSPy aims to optimize the use of language models through a structured and declarative programming approach.